In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import torch as t

In [2]:
def read_U_I(data, file):
    # [user, item]
    U_I = []
    max_user = 0
    max_item = 0
    with open('../datasets/'+ data +'/' + file , 'r') as f:
        lines = f.readlines()
        for i in lines:
            elements = i.split()
            sharer = int(elements[0])
            item = int(elements[1])
            participant = int(elements[2])
            max_user = max(sharer, participant, max_user)
            max_item = max(max_item, item)
            U_I.append([sharer, item])
            U_I.append([participant, item])
    data = np.ones(len(U_I))
    users, items = list(zip(*U_I))
    matrix = csr_matrix((data, (users,items)), shape = (max_user + 1, max_item + 1))

    x = matrix.nonzero()
    data = np.ones(len(x[0]))
    # pdb.set_trace()
    matrix = csr_matrix((data, (list(x[0]), list(x[1]))), shape = (max_user + 1, max_item + 1))
    # matrix = torch.sparse_coo_tensor([list(x[0]), list(x[1])], data, (self.numUsers, self.numItems))
    U_I = []
    for i in range(len(x[0])):
        U_I.append([x[0][i], x[1][i]])


    return U_I, matrix

In [3]:
def read_U_I_U(data, file):
    # [user, item, user]
    U_I_U = []
    with open('../datasets/'+ data +'/' + file , 'r') as f:
        lines = f.readlines()
        for i in lines:
            elements = i.split()
            sharer = int(elements[0])
            item = int(elements[1])
            participant = int(elements[2])
            U_I_U.append([sharer, item, participant])
    return U_I_U

In [19]:
U_I_train, matrix_train = read_U_I('BeiBei', 'train.txt')
U_I_U_test = read_U_I_U('BeiBei', 'test.txt')

In [20]:
U_I_train_dict = {}
for pair in U_I_train:
    u = pair[0]
    i = pair[1]
    if u not in U_I_train_dict:
        U_I_train_dict[u] = [i]
    else:
        U_I_train_dict[u].append(i)
# U_I_test_dict = {}
# for pair in U_I_test:
#     u = pair[0]
#     i = pair[1]
#     if u not in U_I_test_dict:
#         U_I_test_dict[u] = [i]
#     else:
#         U_I_test_dict[u].append(i)

In [21]:
def read_social_relation(data):
    # [user, user]
    social = []
    friendship = {}
    with open('../datasets/'+ data +'/' + 'social_relation.txt','r') as f:
        lines = f.readlines()
        for i in lines:
            elements = i.split()
            u1 = int(elements[0])
            u2 = int(elements[1])
            social.append([u1, u2])
            if u1 not in friendship:
                friendship[u1] = [u2]
            else:
                friendship[u1].append(u2)
            if u2 not in friendship:
                friendship[u2] = [u1]
            else:
                friendship[u2].append(u1)
    for k,v in friendship.items():
        friendship[k] = list(set(v))
    return social, friendship

In [22]:
_, friendship = read_social_relation('brightkite')

In [23]:
result = []
for u_i_u in U_I_U_test:
    u1 = u_i_u[0]
    u2 = u_i_u[2]
    score = F.cosine_similarity(t.tensor(matrix_train[u1].toarray()), t.tensor(matrix_train[u2].toarray())).item()
    result.append(score)

In [24]:
sum(result)/len(result)

0.0404371828025607